In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import syft as sy
import numpy as np
import torch as th
from syft import VirtualMachine
from pathlib import Path
from torchvision import datasets, transforms
from syft.core.plan.plan_builder import PLAN_BUILDER_VM, make_plan, build_plan_inputs, ROOT_CLIENT
from syft.lib.python.collections.ordered_dict import OrderedDict
from syft.lib.python.list import List
from matplotlib import pyplot as plt
from syft import logger
from syft import SyModule, SySequential
logger.remove()

# Dataset

In [3]:
mnist_path = Path.home() / ".pysyft" / "mnist"
mnist_path.mkdir(exist_ok=True, parents=True)

In [4]:
mnist_train = datasets.MNIST(str(mnist_path), train=True, download=True,
               transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]))

mnist_test = datasets.MNIST((mnist_path), train=False, 
              transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]))

In [5]:
train_loader = th.utils.data.DataLoader(mnist_train, batch_size=64*3, shuffle=True, pin_memory=True)
test_loader = th.utils.data.DataLoader(mnist_test, batch_size=1024, shuffle=True, pin_memory=True)

# Define Plan

In [6]:
# !pip install timm
import timm

For nn.Sequential, we need to infer the first parameter / layer that is used.

For user defined blocks, we need to infer the size of the input for a particular block definition

In [7]:
class MySyModuleBlock(SyModule):
    def __init__(self, n_in, n_out, **kwargs):
        super().__init__(**kwargs)
        self.layer = th.nn.Linear(n_in, n_out)
    
    def forward(self, x):
        o1 = self.layer(x)
        return o1

In [8]:
sequential = SySequential(
    MySyModuleBlock(100, 10, input_size=(32,100)),
    MySyModuleBlock(10, 10, input_size=(32,10)),
)

In [9]:
model = timm.create_model('resnet18', pretrained=True)
# model = timm.create_model('resnet18d', pretrained=True)

In [10]:
# timm.list_models(pretrained=True)

In [11]:
class BasicBlock(SyModule):
    
    def __init__(self, strides=None, **kwargs):
        super().__init__(**kwargs)
        if strides is None:
            strides = [()]
        
        self.conv1 = th.nn.Conv2d(64, 64, kernel_size=(3, 3), padding=(1, 1), bias=False)
        self.bn1 = th.nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.act1 = th.nn.ReLU()
        self.conv2 = th.nn.Conv2d(64, 64, kernel_size=(3, 3), padding=(1, 1), bias=False)
        self.bn2 = th.nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.act2 = th.nn.ReLU()
        self.downsample = None
        
    
    def forward(self, x):
        residual = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.bn2(x)

        if self.downsample is not None:
            residual = self.downsample(residual)
        x += residual
        x = self.act2(x)
        return x
        

In [12]:
BasicBlock(strides = 1, input_size=(32, 64, 128, 128))


BasicBlock(
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU()
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU()
)

In [15]:
class ResNet18(SyModule):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = th.nn.Conv2d(3, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False)
        self.bn1 = th.nn.BatchNorm2d(64)
        self.act1 = th.nn.ReLU()
        self.maxpool = th.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        
        for i in range(1,5):
            layer = SySequential(
                BasicBlock(strides = 1, input_size=(32, 64, 128, 128)),
                BasicBlock(strides = 1, input_size=(32, 64, 128, 128))
            )
            setattr(self, f"layer{i}", layer)
        
        
        self.global_pool = th.nn.AdaptivePool2d(1)
        #TODO, Flatten x = x.flatten(1)
        self.fc = th.nn.Linear(in_features=512, out_features=1000, bias=True)
        
        

In [16]:
ResNet18()

TypeError: cannot assign 'syft.lib.python._SyNone' as parameter 'bias' (torch.nn.Parameter or None expected)

In [35]:
# make_dot(y)

In [36]:
# model.state_dict().keys()

In [12]:
def set_params(model, params):
    """happens outside of plan"""
    for p, p_new in zip(model.parameters(), params): p.data = p_new.data

In [13]:
def cross_entropy_loss(logits, targets, batch_size):
    norm_logits = logits - logits.max()
    log_probs = norm_logits - norm_logits.exp().sum(dim=1, keepdim=True).log()
    return -(targets * log_probs).sum() / batch_size

In [14]:
def sgd_step(model, lr=0.1):
    with ROOT_CLIENT.torch.no_grad():
        for p in model.parameters():
            p.data = p.data - lr * p.grad            
            p.grad = th.zeros_like(p.grad.get())

In [15]:
class CNN(th.nn.Sequential):
    def __init__(self, filters, kernel_szs=None, strides=None, bn=True):
        nl = len(filters)-1
        kernel_szs = kernel_szs if kernel_szs is not None else [3] * nl
        strides    = strides if strides is not None else [2] * nl
        layers = [ConvLayer(filters[i], filters[i+1], kernel_szs[i], stride=strides[i]) 
                  for i in range(nl)]
        layers.append(PoolFlatten())
        super().__init__(*layers)
        
    def call(self):
        print("A")

In [46]:
class ConvLayer(th.nn.Sequential):
    def __init__(self, ni, nf, ks=3, stride=1, padding=None):
        padding = (ks-1)//2
            
        conv = th.nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=padding)
        th.nn.init.normal_(conv.bias, 0, 0.01)
        th.nn.init.kaiming_uniform_(conv.weight)
        
        bn = th.nn.BatchNorm2d(nf)
        bn.bias.data.fill_(1e-3)
        bn.weight.data.fill_(1.)
        
        super().__init__(conv, bn, th.nn.ReLU())

In [47]:
class PoolFlatten(th.nn.Sequential):
    def __init__(self):
        super().__init__(th.nn.AdaptiveAvgPool2d(1), th.nn.Flatten())

In [48]:
cnn = CNN([1,16,10])

In [49]:
local_model = CNN([1,16,10])

In [1]:
# Cell
class XResNet(nn.Sequential):
#     @delegates(ResBlock)
    def __init__(self, block, expansion, layers, p=0.0, c_in=3, n_out=1000, stem_szs=(32,32,64),
                 widen=1.0, sa=False, act_cls=defaults.activation, ndim=2, ks=3, stride=2, **kwargs):
        store_attr('block,expansion,act_cls,ndim,ks')
        if ks % 2 == 0: raise Exception('kernel size has to be odd!')
        stem_szs = [c_in, *stem_szs]
        stem = [ConvLayer(stem_szs[i], stem_szs[i+1], ks=ks, stride=stride if i==0 else 1,
                          act_cls=act_cls, ndim=ndim) for i in range(3)]

        block_szs = [int(o*widen) for o in [64,128,256,512] +[256]*(len(layers)-4)]
        block_szs = [64//expansion] + block_szs
        blocks    = self._make_blocks(layers, block_szs, sa, stride, **kwargs)

        super().__init__(
            *stem, MaxPool(ks=ks, stride=stride, padding=ks//2, ndim=ndim),
            *blocks,
            AdaptiveAvgPool(sz=1, ndim=ndim), Flatten(), nn.Dropout(p),
            nn.Linear(block_szs[-1]*expansion, n_out),
        )
        init_cnn(self)

    def _make_blocks(self, layers, block_szs, sa, stride, **kwargs):
        return [self._make_layer(ni=block_szs[i], nf=block_szs[i+1], blocks=l,
                                 stride=1 if i==0 else stride, sa=sa and i==len(layers)-4, **kwargs)
                for i,l in enumerate(layers)]

    def _make_layer(self, ni, nf, blocks, stride, sa, **kwargs):
        return nn.Sequential(
            *[self.block(self.expansion, ni if i==0 else nf, nf, stride=stride if i==0 else 1,
                      sa=sa and i==(blocks-1), act_cls=self.act_cls, ndim=self.ndim, ks=self.ks, **kwargs)
              for i in range(blocks)])

# Cell
def _xresnet(pretrained, expansion, layers, **kwargs):
    # TODO pretrain all sizes. Currently will fail with non-xrn50
    url = 'https://s3.amazonaws.com/fast-ai-modelzoo/xrn50_940.pth'
    res = XResNet(ResBlock, expansion, layers, **kwargs)
    if pretrained: res.load_state_dict(load_state_dict_from_url(url, map_location='cpu')['model'], strict=False)
    return res

NameError: name 'nn' is not defined